In [1]:
from api.db import stat_db, crawl_db, CrawlCollections, StatCollections, relation_db, RelationCollections, DESCENDING, ASCENDING
import time
import math
import re
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:

comments = pd.DataFrame([item['data'] for item in crawl_db[CrawlCollections.BTT_COMMENT].find({'data.link_id': '2207367'})],
                                columns=['time'])
comments.time = comments['time'].apply(lambda x: datetime.fromtimestamp(x))
comments.set_index(comments.time, inplace=True)
comments['num'] = 1
comments.drop('time', axis=1)
temp = comments.resample('D').sum().fillna(0)
temp.reset_index().to_json(orient='values')


In [ ]:
comments = crawl_db[CrawlCollections.BTT_COMMENT].find()
select_comments = []
for comment in comments:
    try:
        text = comment['data']['content']
        if re.search(r'\bsmart contract\b', text, re.I) is not None:
            select_comments.append(comment)
    except Exception:
        pass

In [12]:

df = pd.DataFrame([item['data'] for item in select_comments], columns=['time', 'activity', 'author', 'link_id', 'user_id', 'grade'])
df['num'] = 1
group = df.groupby(['author', 'grade'], as_index=False)
grouped_df = group.agg({'time': np.min, 'num': np.sum, 'activity': np.max})
d = grouped_df['grade'].astype('category')
grouped_df['time'] = grouped_df['time'] * 1000
max = grouped_df['num'].max()
min = grouped_df['num'].min()
grouped_df['num'] = grouped_df['num'].map(lambda x: np.floor((x - min) / (max - min) * 48 + 2))
activity = grouped_df['activity']
time = grouped_df['time']
grouped_df.drop(labels=['activity'], axis=1,inplace=True)
grouped_df.drop(labels=['time'], axis=1,inplace = True)
grouped_df.insert(0, 'activity', activity)
grouped_df.insert(0, 'time', time)
top30 = grouped_df.sort_values(by='time')[:30]
top30_json = top30.to_json(orient='values')
json_arr = []
for grade in d.cat.categories:
    df = grouped_df[grouped_df['grade'] == grade]
    json = df.to_json(orient='values')
    json_arr.append({'data': json, 'grade': grade})   



In [16]:
df = pd.DataFrame([item['data'] for item in select_comments], columns=['time', 'link_id'])
relation_btt = relation_db[RelationCollections.RELATION_CURRENCY_BTT].find({'ann': True})
link_replies = []
for item in relation_btt:
    link = crawl_db[CrawlCollections.BTT_LINK].find_one({'data.id': item['topic_id']})
    if link is not None:
        link_replies.append({'topic_id': link['data']['id'], 'replies': link['data']['replies'], 'currency_name': item['currency_name']})
df2 = pd.DataFrame([item for item in link_replies], columns=['currency_name', 'topic_id', 'replies'])
df2['num'] = 1
merge_df = pd.merge(df, df2, how='left', left_on='link_id', right_on='topic_id')


merge_df2 = merge_df.dropna()

grouped_df = merge_df2.groupby(['currency_name', 'link_id']).agg({'time': np.min, 'num': np.sum, 'replies': np.max})

grouped_df2 = grouped_df.reset_index()
grouped_df2.head()
json_arr2 = grouped_df2.to_json(orient='values') 
stat_db["btt_keyword_stat"].find_one_and_update({'keyword': 'smart contract'},
                                                {'$set': {'project_data': json_arr2}},
                                                upsert=True)


统计

In [ ]:


relation_btt = relation_db[RelationCollections.RELATION_CURRENCY_BTT].find({'ann': True})
relation_df = pd.DataFrame([item for item in relation_btt], columns=['topic_id', 'currency_name'])

# 获取数据库中所有的用户
user_list = crawl_db[CrawlCollections.BTT_COMMENT].aggregate([{'$group': {'_id': '$data.user_id'}}])
for user in user_list:
    user_id = user['_id']

    # 从数据库中查找用户分析过的数据，如果不为空则跳出循环
    stat_data = stat_db[StatCollections.BTT_USER_HISTORY_STAT].find_one({'_id': user_id})
    if stat_data is not None:
        break

    # 从数据库中加载该用户的所有评论
    df = pd.DataFrame(
        [item['data'] for item in crawl_db[CrawlCollections.BTT_COMMENT].find({'data.user_id': user_id})],
        columns=['time', 'author', 'user_id', 'link_id'])

    # 将关系data frame和
    merged_df = pd.merge(df, relation_df, left_on='link_id', right_on='topic_id', how='left')
    clean_merged_df = merged_df.dropna().drop('topic_id', axis=1)

    # 将数据按照currency name分组，获取每个currency的评论数量
    clean_merged_df['num'] = 1
    grouped_df = clean_merged_df.groupby(['link_id', 'currency_name'], as_index=False).agg({'num': np.sum})
    general_data = grouped_df.to_json(orient='values')

    # 将用户的足迹按照currency name 分类, 获取评论分布图
    arr_data = []
    clean_merged_df['time'] = clean_merged_df['time'].apply(lambda x: datetime.fromtimestamp(x))
    clean_merged_df.set_index(clean_merged_df.time, inplace=True)
    clean_merged_df.drop('time', axis=1)
    d = clean_merged_df['currency_name'].astype('category')
    for currency_name in d.cat.categories:
        temp_df = clean_merged_df[clean_merged_df['currency_name'] == currency_name]
        resampled_df = temp_df.resample('D').sum().fillna(0)
        json_data = resampled_df.reset_index().to_json(orient='values')
        arr_data.append({currency_name: json_data})


## 找到btt中评论最多的前100人


In [ ]:

user_list = crawl_db[CrawlCollections.BTT_COMMENT].aggregate([{'$group': {'_id': '$data.user_id', 'num': {'$sum': 1}}}])
user_df = pd.DataFrame([item for item in user_list], columns=['_id', 'num'])
user_df = user_df.sort_values(by='num', ascending=False)[:100]
for index, row in user_df.iterrows():
    stat_db['btt_kol'].find_one_and_update({'_id': row['_id']}, {'$set': {'user_id': row['_id'], 'num': row['num']}}, upsert=True)


## 按照时间点分析btt的评论情况


In [ ]:
t = 1447084800
one_day = 3600 * 24

cs = crawl_db[CrawlCollections.BTT_COMMENT].find({'data.time': {'$gte': t, '$lte': t+one_day}})
df = pd.DataFrame([i[''] for i in cs])


In [ ]:
crawl_db[RelationCollections.RELATION_CURRENCY_BTT].find()
